In [113]:
def lzw_encode_max(input_file, max_dict_size):
    # Inicializa o dicionário com todos os símbolos possíveis
    dict_size = 256
    dictionary = {chr(i): i for i in range(dict_size)}
    #full_encode = []
    # Inicializa a sequência atual como vazia
    current_sequence = ""

    # Abre o arquivo de entrada para leitura e o arquivo de saída para escrita
    with open(input_file, "rb") as file, open("output.bin", "wb") as output_file:
        # Codifica cada símbolo da entrada
        for symbol in file.read():
            # Acrescenta o próximo símbolo à sequência atual
            next_sequence = current_sequence + chr(symbol)

            # Se a próxima sequência estiver no dicionário, continua acrescentando símbolos
            # até chegar a uma sequência que não esteja no dicionário
            if next_sequence in dictionary:
                current_sequence = next_sequence
            else:
                # Codifica a sequência atual e adiciona a nova sequência ao dicionário
                encoded_sequence = dictionary[current_sequence]
                #full_encode.append(bin(encoded_sequence)[2:].zfill(16))
                output_file.write(encoded_sequence.to_bytes(2, byteorder="big"))
                
                if dict_size < 2 ** max_dict_size:
                    dictionary[next_sequence] = dict_size
                    dict_size += 1
                # Começa uma nova sequência com o último símbolo lido
                current_sequence = chr(symbol)

        # Codifica a última sequência atual
        encoded_sequence = dictionary[current_sequence]
        #full_encode.append(bin(encoded_sequence)[2:].zfill(16))
        output_file.write(encoded_sequence.to_bytes(2, byteorder="big"))
        #return full_encode, dictionary


In [114]:
lzw_encode_max(r'C:\Users\franklin.coelho\Documents\encode-decode-lzw\files\corpus16MB.txt', 10)

In [115]:
def lzw_decode_max(input_file, max_dict_size):
    # Inicializa o dicionário com todos os símbolos possíveis
    dict_size = 256
    dictionary = {i: chr(i) for i in range(dict_size)}

    # Lê o arquivo de entrada e decodifica os códigos LZW
    with open(input_file, "rb") as file, open("output.txt", "w") as output_file:
        # Lê os primeiros dois bytes do arquivo como o primeiro código LZW
        current_code = int.from_bytes(file.read(2), byteorder="big")
        # Decodifica o primeiro código LZW como um caractere
        current_sequence = dictionary[current_code]
        # Escreve o primeiro caractere decodificado no arquivo de saída
        output_file.write(current_sequence)

        # Continua decodificando os códigos LZW até que o arquivo de entrada seja lido completamente
        while True:
            # Lê o próximo código LZW do arquivo
            next_code = file.read(2)
            # Se o final do arquivo for alcançado, sai do loop
            if not next_code:
                break
            # Converte o próximo código LZW em um inteiro
            next_code = int.from_bytes(next_code, byteorder="big")
            # Se o próximo código LZW estiver no dicionário, decodifica-o
            if next_code in dictionary:
                next_sequence = dictionary[next_code]
            # Senão, decodifica a sequência atual acrescida do primeiro caractere da próxima sequência
            else:
                next_sequence = current_sequence + current_sequence[0]
            # Escreve a sequência decodificada no arquivo de saída
            output_file.write(next_sequence)
            # Adiciona a próxima sequência ao dicionário
            dictionary[dict_size] = current_sequence + next_sequence[0]
            dict_size += 1
            # Atualiza a sequência atual para a próxima sequência
            current_sequence = next_sequence
            # Atualiza o código LZW atual para o próximo código LZW
            current_code = next_code


In [116]:
lzw_decode_max(r'C:\Users\franklin.coelho\Documents\encode-decode-lzw\output.bin', 10)